<a href="https://colab.research.google.com/github/Anilesh05/Big_Data_Laboratory/blob/main/14_find_number_of_product_sold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download and install hadoop

In [ ]:
!apt-get install openjdk-8-jdk
!wget https://dlcdn.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
!tar fx hadoop-3.3.6.tar.gz
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/content/hadoop-3.3.6"
!ln -s /content/hadoop-3.3.6/bin/* /usr/bin

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java libatk-wrapper-java-jni libfontenc1
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libice-dev librsvg2-common
  libsm-dev libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-8-jdk-headless openjdk-8-jre
  openjdk-8-jre-headless x11-utils
Suggested packages:
  gvfs libice-doc libsm-doc libxt-doc openjdk-8-demo openjdk-8-source visualvm libnss-mdns
  fonts-nanum fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei
  fonts-indic mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java libatk-wrapper-java-jni libfontenc1
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libice-dev librsvg2-common
  libsm-dev libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-

## Create mapper.py

In [ ]:
%%writefile mapper.py
#!/usr/bin/env python

import sys

# Input comes from STDIN (standard input)
for line in sys.stdin:
    # Remove leading and trailing whitespace
    line = line.strip()
    # Split the line into fields
    fields = line.split(',')

    # Extract country and product
    country = fields[6]
    product = fields[1]

    # Emit key-value pair for product and country
    print('%s\t%s' % (product, country))


Writing mapper.py


## Create reducer.py

In [ ]:
%%writefile reducer.py
#!/usr/bin/env python

import sys

current_product = None
current_country = None
current_count = 0

# Print column names
print('{:15} {:<16} {:<15}'.format("product","Country","Count"))
print()


# Input comes from STDIN
for line in sys.stdin:
    # Remove leading and trailing whitespace
    line = line.strip()

    # Parse the input we got from mapper.py
    product, country = line.split('\t', 1)

    # If the current product and country are equal to the product and country we just read,
    # increment the count for this product
    if current_product == product and current_country == country:
        current_count += 1
    else:
        # If it's not the first product or country (i.e., current_product is not None),
        # print out the count for the previous product and country
        if current_product:
            print(f"{current_product:<15} {current_country:<16} {current_count}")
        current_count = 1
        current_product, current_country = product, country

# Output the last product and country
if current_product:
      print(f"{current_product:<15} {current_country:<16} {current_count}")


Writing reducer.py


## Create a input directory

In [ ]:
!mkdir input

In [ ]:
%%writefile input/input.txt
2024-04-01,Product_A,50,John,Dallas,Texas,USA,2023-01-01,2024-03-30,32.7767,-96.7970
2024-04-01,Product_B,30,Jane,New York,New York,USA,2022-06-15,2024-03-31,40.7128,-74.0060
2024-04-02,Product_A,50,John,Los Angeles,California,USA,2021-12-10,2024-04-01,34.0522,-118.2437
2024-04-02,Product_C,20,Alice,Paris,,France,2023-05-20,2024-03-29,48.8566,2.3522
2024-04-03,Product_A,50,John,London,,UK,2023-02-14,2024-04-02,51.5074,-0.1278
2024-04-03,Product_B,30,Bob,Sydney,NSW,Australia,2022-09-30,2024-04-01,-33.8688,151.2093
2024-04-03,Product_A,50,John,New Delhi,,India,2023-04-05,2024-03-31,28.6139,77.2090
2024-04-04,Product_B,30,Bob,Tokyo,,Japan,2022-12-25,2024-03-30,35.6895,139.6917
2024-04-05,Product_A,50,John,Beijing,,China,2023-01-20,2024-04-02,39.9042,116.4074
2024-04-05,Product_C,20,Emma,Berlin,,Germany,2022-08-15,2024-03-29,52.5200,13.4050
2024-04-06,Product_B,30,Bob,Rio de Janeiro,,Brazil,2022-11-10,2024-04-02,-22.9068,-43.1729
2024-04-07,Product_A,50,John,Moscow,,Russia,2023-03-03,2024-04-01,55.7558,37.6176
2024-04-08,Product_C,20,Emma,Mexico City,,Mexico,2022-07-05,2024-03-30,19.4326,-99.1332
2024-04-09,Product_B,30,Bob,Istanbul,,Turkey,2022-10-01,2024-04-03,41.0082,28.9784
2024-04-10,Product_A,50,John,Cairo,,Egypt,2023-06-10,2024-04-01,30.0444,31.2357
2024-04-11,Product_B,30,Bob,Buenos Aires,,Argentina,2023-01-30,2024-03-31,-34.6037,-58.3816
2024-04-12,Product_A,50,John,Lima,,Peru,2023-04-20,2024-04-03,-12.0464,-77.0428
2024-04-13,Product_C,20,Emma,Bangkok,,Thailand,2022-12-01,2024-04-02,13.7563,100.5018
2024-04-14,Product_B,30,Bob,Seoul,,South Korea,2023-02-14,2024-03-29,37.5665,126.9780
2024-04-15,Product_A,50,John,Mumbai,,India,2023-07-10,2024-04-01,19.0760,72.8777
2024-04-15,Product_B,30,Bob,Lagos,,Nigeria,2023-03-15,2024-04-03,6.5244,3.3792
2024-04-16,Product_C,20,Emma,Kinshasa,,DR Congo,2023-01-10,2024-04-02,-4.4419,15.2663
2024-04-17,Product_A,50,John,Karachi,,Pakistan,2023-08-25,2024-04-01,24.8607,67.0011
2024-04-18,Product_B,30,Bob,Shanghai,,China,2023-04-30,2024-04-03,31.2304,121.4737
2024-04-19,Product_A,50,John,Moscow,,Russia,2023-06-01,2024-04-01,55.7558,37.6176
2024-04-20,Product_C,20,Emma,Beijing,,China,2023-02-18,2024-03-29,39.9042,116.4074
2024-04-21,Product_B,30,Bob,Berlin,,Germany,2023-01-15,2024-04-02,52.5200,13.4050
2024-04-22,Product_A,50,John,Rio de Janeiro,,Brazil,2023-05-05,2024-04-01,-22.9068,-43.1729
2024-04-23,Product_C,20,Emma,Mexico City,,Mexico,2023-03-20,2024-04-03,19.4326,-99.1332
2024-04-24,Product_B,30,Bob,Tokyo,,Japan,2023-07-30,2024-04-01,35.6895,139.6917
2024-04-25,Product_A,50,John,Paris,,France,2023-10-10,2024-04-02,48.8566,2.3522
2024-04-26,Product_C,20,Emma,Sydney,NSW,Australia,2023-04-25,2024-04-03,-33.8688,151.2093
2024-04-27,Product_B,30,Bob,London,,UK,2023-01-05,2024-04-01,51.5074,-0.1278
2024-04-28,Product_A,50,John,New York,New York,USA,2022-09-01,2024-04-02,40.7128,-74.0060
2024-04-29,Product_B,30,Bob,Dallas,Texas,USA,2022-05-20,2024-04-03,32.7767,-96.7970
2024-04-30,Product_C,20,Emma,Los Angeles,California,USA,2023-03-10,2024-04-01,34.0522,-118.2437

Writing input/input.txt


In [ ]:
!hadoop jar /content/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar \
    -files mapper.py,reducer.py \
    -mapper mapper.py \
    -reducer reducer.py \
    -input input \
    -output output

2024-04-14 16:25:25,770 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2024-04-14 16:25:25,960 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2024-04-14 16:25:25,960 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2024-04-14 16:25:25,985 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2024-04-14 16:25:26,382 INFO mapred.FileInputFormat: Total input files to process : 1
2024-04-14 16:25:26,405 INFO mapreduce.JobSubmitter: number of splits:1
2024-04-14 16:25:26,574 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1568991692_0001
2024-04-14 16:25:26,574 INFO mapreduce.JobSubmitter: Executing with tokens: []
2024-04-14 16:25:26,968 INFO mapred.LocalDistributedCacheManager: Localized file:/content/mapper.py as file:/tmp/hadoop-root/mapred/local/job_local1568991692_0001_73202828-fc1c-453d-9738-6d1fde84c5aa/mapper.py
2024-04-14 16:25:26,986 INFO mapred.LocalDistributedCacheMa

In [ ]:
!cat output/part-00000

product         Country          Count          	
	
Product_A       China            1	
Product_A       India            1	
Product_A       UK               1	
Product_A       USA              2	
Product_A       Brazil           1	
Product_A       Russia           1	
Product_A       Pakistan         1	
Product_A       India            1	
Product_A       USA              1	
Product_A       Peru             1	
Product_A       Egypt            1	
Product_A       France           1	
Product_A       Russia           1	
Product_B       Japan            1	
Product_B       USA              1	
Product_B       Australia        1	
Product_B       Nigeria          1	
Product_B       Japan            1	
Product_B       South Korea      1	
Product_B       USA              1	
Product_B       Turkey           1	
Product_B       Germany          1	
Product_B       UK               1	
Product_B       Brazil           1	
Product_B       China            1	
Product_B       Argentina        1	
Product_C   